## RedisやKeyDBによるngramの集計コード（マルチプロセスの参考例）

In [1]:
import redis
pool = redis.ConnectionPool(host='localhost', port=6379, db=0)
client = redis.StrictRedis(connection_pool=pool)

In [ ]:
import os
import tarfile
import glob
import gzip
import json
import pandas as pd
from tqdm import tqdm
import time
from pathos.multiprocessing import ProcessingPool
import random
random.seed(777)
#1_ngrammakerで作成した資料ID毎のngram集計情報のjsonが格納されたディレクトリを指定
INPUTDIR="/data/ngramviewer/ngramdata/"
cnt=0
inputgzfilelist=glob.glob(os.path.join(INPUTDIR,"*.json.gz"))
random.shuffle(inputgzfilelist)
pidlist=[os.path.basename(inputgzpath).split(".")[0] for inputgzpath in  inputgzfilelist]
dfnew=pd.DataFrame({"PID":pidlist,"inputgzpath":inputgzfilelist})
print(dfnew.shape)

In [11]:
def dataloader(tmpdf):
    pipe = client.pipeline()
    for index,row in tmpdf.iterrows():
        year=row["出版年(W3CDTF)"]
        inputgzpath=row["inputgzpath"]
        pid=os.path.basename(inputgzpath).split(".")[0]
        pipe.rpush("masterpidlist",pid)
        with gzip.open(inputgzpath, mode='rt') as fp:
            jsonobj=json.load(fp)
            for k,v in jsonobj.items():
                try:
                    k=str(k)
                    k=k.replace(" ","")
                    pipe.hincrby(k,year,v)
                except:
                    pass
    pipe.execute()
    return 0

In [ ]:
ppool = ProcessingPool(nodes=6)#6プロセスの並列で投入する
collen=dfnew.shape[0]
targetdflist=[]
for i in tqdm(range(collen//100+1)):#1プロセスあたり100冊以内になるように分割
    targetdflist.append(dfnew[i*100:(i+1)*100])
result = ppool.map(lambda x: dataloader(x),targetdflist)